In [0]:
%run ./00_config

In [0]:
from pyspark.sql.functions import col, to_date, count, sum as _sum, avg, round, unix_timestamp, coalesce, lit

try:
    print("🥇 Iniciando procesamiento de la Capa Gold (KPIs Analíticos)...")
    
    # 1. LECTURA DE CAPA SILVER
    df_silver = spark.table(SILVER_TAXI_TABLE)
    
    # 2. TRANSFORMACIONES PREVIAS PARA KPIs
    # Extraemos la fecha para poder hacer la agrupación diaria
    df_kpi_base = df_silver.withColumn("trip_date", to_date(col("pickup_datetime")))
    
    # Calculamos la duración del viaje en minutos
    df_kpi_base = df_kpi_base.withColumn(
        "trip_duration_mins", 
        (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) / 60
    )
    
    # Ingreso total por viaje (Venta + Fees + Impuestos)
    df_kpi_base = df_kpi_base.withColumn(
        "total_revenue",
        coalesce(col("base_passenger_fare"), lit(0)) +
        coalesce(col("tolls"), lit(0)) +
        coalesce(col("sales_tax"), lit(0)) +
        coalesce(col("congestion_surcharge"), lit(0)) +
        coalesce(col("airport_fee"), lit(0)) +
        coalesce(col("tips"), lit(0)) +
        coalesce(col("bcf"), lit(0))
    )
    
    # 3. AGREGACIÓN DIARIA (Creación de KPIs)
    print("📊 Calculando KPIs diarios...")
    df_gold = df_kpi_base.groupBy("trip_date").agg(
        # Ingresos totales diarios
        round(_sum("total_revenue"), 2).alias("ingresos_totales"),
        
        # Tiempo promedio del viaje diario (en minutos)
        round(avg("trip_duration_mins"), 2).alias("tiempo_promedio_viaje_min"),
        
        # Distancia promedio del viaje diario
        round(avg("trip_miles"), 2).alias("distancia_promedio_viaje_millas"),
        
        # Viajes promedio por hora (Total de viajes en el día / 24 horas)
        round((count("*") / 24), 2).alias("viajes_promedio_por_hora")
    ).orderBy("trip_date")
    
    # 4. GUARDADO EN CAPA GOLD
    print(f"💾 Guardando KPIs en la tabla: {GOLD_KPI_TABLE}...")
    df_gold.write.mode("overwrite") \
                 .option("overwriteSchema", "true") \
                 .saveAsTable(GOLD_KPI_TABLE)
                 
    print("✅ Capa Gold completada con éxito. ¡Métricas listas para negocio!")
    
    # Mostramos los resultados finales
    display(df_gold)

except Exception as e:
    print(f"❌ Error en Capa Gold: {str(e)}")